# YRoration (YGate) - ML Models

## Configuration

Pulse and fidelity computation are computation intensive (slow).

In [1]:
PULSE_ENABLED = False
FIDELITY_ENABLED = True

Parameters are stored in a shared script.

In [2]:
from parameters import *

Enable seeding for reproducibility of the training.

## Library

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras_sequential_ascii import keras2ascii
import os
import numpy as np
import pandas as pd

import sys
sys.path.append('..')

from utils.helpers import *

# Disable some console warnings
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2023-11-04 18:00:53.326850: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Check GPU

## Dataset

### Load

In [4]:
x_train = np.load('./data/yrotation/{}/x_train.npy'.format(CSV_CONFIG))
x_val = np.load('./data/yrotation/{}/x_val.npy'.format(CSV_CONFIG))
x_test = np.load('./data/yrotation/{}/x_test.npy'.format(CSV_CONFIG))
y_train = np.load('./data/yrotation/{}/y_train.npy'.format(CSV_CONFIG))
y_val = np.load('./data/yrotation/{}/y_val.npy'.format(CSV_CONFIG))
y_test = np.load('./data/yrotation/{}/y_test.npy'.format(CSV_CONFIG))

FileNotFoundError: [Errno 2] No such file or directory: './data/yrotation/D1_10_Pmin_200/x_train.npy'

In [ ]:
N_ANGLES = x_train.shape[1]
N_PARAMS = y_train.shape[1]

In [ ]:
print('---------------------------------------')
print('- Pulse parameters (y) #', N_PARAMS) # ASSUMING 1 BETA VALUE (ANGLE ALONG Y AXIS)
print('- Gate parameters  (x) #', N_ANGLES) # ASSUMING 1 BETA VALUE (ANGLES ALONG Y AXIS)
print('---------------------------------------')

### Evaluation

<span style="color:red">ATTENTION: Measuring fidelity on the entire train, validation, and test sets could be time consuming. Use `limit` parameter in case.</span>

## MLP

### Create model

Classical multi-layer perceptrons: 1 (input), 12 or 20 (outputs), and a certain number of hidden layers.

<span style="background-color:orange">These MLPs may be either oversized or undersized for the final hardware implementation and/or fidelity, but it is a starting point. We can use KerasTuner or manually tune the model to less/more layers/neurons.</span>

Choose one of the following definitions.

<font color='red'>These parameters are now defined in parameters.py</font>

**Large**

**Medium**

**Small**

In [ ]:
def sine_activation(x):
    return tf.math.sin(x)

<span style="background-color:orange">ATTENTION: Replaced relu with sin activation function.</span>

N_PARAMS = np.load('./data/ygate/N_PARAMS.npy')

In [ ]:
model = Sequential()
model.add(Input(shape=(1,), name='input1'))
for i, n in enumerate(NEURONS_PER_LAYER):
    model.add(Dense(n, name='fc{}'.format(i), kernel_initializer='lecun_uniform'))
    model.add(Activation(activation=sine_activation, name='sine{}'.format(i)))
model.add(Dense(N_PARAMS, name='fc{}'.format(len(NEURONS_PER_LAYER)), kernel_initializer='lecun_uniform'))

In [ ]:
model.summary()

In [ ]:
keras2ascii(model)

In [ ]:
model_id = MODEL_ID_PREFIX + get_basic_id(model)
print(model_id)

In [ ]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.0001),
    loss='mean_squared_error',#tf.losses.MeanSquaredError()
    metrics=[
        'mean_squared_error' #tf.losses.MeanSquaredError()
        #tf.metrics.MeanSquaredLogarithmicError(),
        #tf.metrics.MeanAbsolutePercentageError()
    ])

### Training

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

In [ ]:
best_model_file = PREFIX + '/best_keras_model.h5'
last_model_file = PREFIX + '/last_keras_model.h5'

Enable training (`train_and_save`) to overwrite the model file.

In [ ]:
train_and_save = True

We use Adam optimizer, minimize the Mean Squared Error, and early stop.

Load the saved best model and use it from now on.

In [ ]:
model = load_model(best_model_file, custom_objects={'sine_activation': sine_activation})
#model = load_model(best_model_file)

### Evaluation

Although we may plot and print many metrics, we focus only on **Mean Squared Error (MSE).**

Plot training history.

In [ ]:
from utils.metrics import plot_metrics
plot_metrics(PREFIX,
             framework='Keras',
             save=train_and_save,
             show_all=False, # show MSLE and MAPE, in addition to MSE
             history=history)

Measure and print metrics.

In [ ]:
#mse, msle, mape = model.evaluate(x_test, y_test)
mse = model.evaluate(x_test, y_test)[0]
msle, mape = None, None

In [ ]:
mse = model.evaluate(x_val, y_val)[0]
msle, mape = None, None

In [ ]:
y_keras = model.predict(x_test)

In [ ]:
model.summary()

Plot animation of a sorted test set along with its predictions

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML
import matplotlib.pyplot as plt

# Sort the test values by beta

XY_actual = np.concatenate((x_test, y_test), axis=1)
XY_pred = np.concatenate((x_test, y_keras), axis=1)

#XY_actual = sort_array(XY_actual)
#XY_pred = sort_array(XY_pred)

XY_actual = XY_actual[XY_actual[:,0].argsort()]

XY_pred = XY_pred[XY_pred[:,0].argsort()]

#print(XY_actual)
#print(XY_pred)

## Needs ffmpeg. Install using "conda install -c conda-forge ffmpeg"
start_idx = 0
num_frames = 2000
frame_interval_ms = 100

# get x and y
x = np.arange(1, N_PARAMS+1, 1)
#y = y_test[start_idx:start_idx+num_frames,:]
#y_pred = y_keras[start_idx:start_idx+num_frames,:]
#beta = x_test[start_idx:start_idx+num_frames,:]
y = XY_actual[start_idx:start_idx+num_frames,N_ANGLES:]
beta = XY_actual[start_idx:start_idx+num_frames,:N_ANGLES]
y_pred = XY_pred[start_idx:start_idx+num_frames,N_ANGLES:]

print(y.shape)
print(beta.shape)
print(y_pred.shape)

# create a figure and an axis object
plt.ioff()
fig, ax = plt.subplots()

# set labels and plot limits
plt.ylabel('Pulse parameter values')
plt.xlabel('Pulse parameters (α[1:{}])'.format(N_PARAMS))
#plt.ylim([min_pparam_value, max_pparam_value*5/4])
plt.xticks(x, x)

# initialize an empty line object
line, = ax.plot([], [], marker='o', linestyle='--', color='r', label='actual')
line_pr, = ax.plot([], [], marker='o', linestyle='--', color='g', label='predicted')
ax.legend()

# create a text object inside the axes
title = ax.text(0.05, 0.9, "", transform=ax.transAxes)

# define a function that updates the line object for each frame
def animate(i):
    title.set_text(f"Beta: {beta[i]}")
    # update line data
    line.set_data(x, y[i])
    line_pr.set_data(x, y_pred[i])
    return line,

# create an animation object using FuncAnimation
anim = animation.FuncAnimation(fig, animate,
                              frames=num_frames,
                              interval=frame_interval_ms,
                              repeat=False)

# show the animation
%matplotlib inline
HTML(anim.to_html5_video())

In [ ]:
## Added Code to check visually the difference in signals from the model and the actual sample

%matplotlib inline
import matplotlib.pyplot as plt
    
entry_id = 0
outside = 0

x_new = []
y_new = []
while (entry_id < x_test.shape[0]):
    angle = x_test[entry_id:entry_id+1][0][0]
    if (angle > 3 or angle < -3):
        outside += 1
    else:
        x_new.append(x_test[entry_id])
        y_new.append(y_keras[entry_id])
    entry_id += 1
entry_id -= 1
x_new = np.array(x_new)
y_new = np.array(y_new)
print(outside)
x = np.arange(1, N_PARAMS+1, 1)
y = y_test[entry_id:entry_id+1,:N_PARAMS][0]
y_pr = y_keras[entry_id:entry_id+1,:N_PARAMS][0]

plt.plot(x, y, marker='o', linestyle='--', color='r', label='actual') 
plt.plot(x, y_pr, marker='o', linestyle='--', color='g', label='predicted') 
plt.legend(loc="upper right")
plt.xticks(x, x)
angle = x_test[entry_id:entry_id+1][0][0]
plt.title('Gate parameter (angle) {}'.format(angle))
plt.ylabel('Amplitude')
plt.xlabel('Pulse parameters')
plt.show()

In [ ]:
import random
hash_id = random.getrandbits(32)

if FIDELITY_ENABLED:
    ygate_fidelity_keras = get_ygate_fidelity(x_test, y_keras,
                                              limit=x_test.shape[0],
                                              config_template=CONFIG_TEMPLATE_JSON,
                                              pulse_data_path='/tmp/ygate_{:x}.csv'.format(hash_id),
                                              output_objf_path='/tmp/ygate_fidelity_{:x}.csv'.format(hash_id))
else:
    ygate_fidelity_keras = None

Save metrics values to file for future reference.

In [ ]:
metrics_filename = PREFIX + '/metrics.csv'

# You can disable the writing if necessary.
write_metrics_csv = True

In [ ]:
from utils.metrics import write_metrics
if write_metrics_csv:
    write_metrics(metrics_filename, 'Keras', model_id, mse, msle, mape, ygate_fidelity_keras)

In [ ]:
from utils.metrics import print_metrics
print_metrics(metrics_filename)

### Save traces

These traces can be sared with quantum experts.

In [ ]:
data_y_test = pd.DataFrame(y_test)
data_y_test = data_y_test.reset_index(drop=True)

data_x_test = pd.DataFrame(x_test)
data_x_test = data_x_test.reset_index(drop=True)

data_test = pd.concat([data_y_test, data_x_test], axis=1)

data_test.to_csv('reference.csv')

In [ ]:
data_y_keras = pd.DataFrame(y_keras)
data_y_keras = data_y_keras.reset_index(drop=True)

data_x_test = pd.DataFrame(x_test)
data_x_test = data_x_test.reset_index(drop=True)

data_keras = pd.concat([data_y_keras, data_x_test], axis=1)

data_keras.to_csv('keras_results.csv')